### Библиотеки

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import json
import pickle
import os

from tqdm.auto import tqdm, trange
from IPython.display import display

from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from datasets import Dataset
from transformers import TrainingArguments
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from trl import DPOTrainer

from collections import defaultdict
import gc
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

/home/user/conda/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


'cuda'

### Полезные функции

In [2]:
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
from scipy.stats import entropy
from collections import defaultdict

def token_entropy(generations, tokenizer):
    stats = defaultdict(int)
    num_tokens = 0
    for example in tqdm(generations, desc="Evaluating"):
        tokens = tokenizer.encode(example)
        for t in tokens:
            if t == tokenizer.pad_token_id:
                continue
            stats[t] += 1
            num_tokens += 1
    for k in stats.keys():
        stats[k] /= num_tokens
    return entropy(list(stats.values()))

Кажется, что эта функция и так достаточно неплохо оценивает разнообразие, я и не знаю, что можно здесь придумать лучше энтропии, ведь это буквально то, что она должна делать по определению - оценивать разнообразие. Можно было бы попробовать подсчитать в лоб долю испольхованных слов словаря или долю разных токенов в предсказании, но это какие-то ни на чём не основанные вещи

### Часть 1

#### 1.1 Проверка генерации

Ниже пройдёмся по всем пунктам, что были в первой части. Сперва нужно достать sft-модель и разобраться, как она работает. Для этого достаточно ознакомиться с интерфейсом с huggingface

In [4]:
model_name = "lvwerra/gpt2-imdb"

tokenizer = GPT2Tokenizer.from_pretrained(
    model_name, padding_side='left',
)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(
    model_name, pad_token_id=tokenizer.pad_token_id,
).to(device)

Ниже простенькая функция генерации. Параметры я подобрал такие, чтобы получалось что-то более-менее осмысленное, но не слишком длинное, а то долго обучаться. Сразу же добавил дефолтные опции для генерации положительных отзывов

In [5]:
def generate_from_input(
    inp="",
    debug=False,
    naive_guidance=None,
    **generation_config
):
    guidance_dict = {
        "positive": f"Good Review: {inp}",
        "negative": f"Negative Review: {inp}",
        None: "..." if inp == "" else inp
    }
    inp = guidance_dict[naive_guidance].rstrip()
    input_ids = tokenizer.encode(
        inp, return_tensors='pt',
        add_special_tokens=False,
        padding=True
    ).to(device)
    beam_output = model.generate(
        input_ids, do_sample=True, **generation_config
    ).to(device)
    output = tokenizer.decode(
        beam_output[0], skip_special_tokens=False,
        clean_up_tokenization_spaces=False,
    )
    if debug:
        print(output)
    return ".".join(output.split(".")[:-1]) + "."

generation_config = dict(
    top_k=50,
    num_beams=5,
    max_length=100,
    early_stopping=True,
    no_repeat_ngram_size=2,
    renormalize_logits=True
)

In [10]:
generate_from_input()

'... I am very glad I bought this one.'

In [14]:
generate_from_input(naive_guidance="positive")

'Good Review: After reading all the other reviews , i want to give credit where credit is due.'

Кажется, всё работает. Дальше нужна модель, которая будет эти отзывы оценивать, тоже из задания

In [6]:
model_name = "lvwerra/distilbert-imdb"

clf_tokenizer = DistilBertTokenizer.from_pretrained(model_name)
clf = DistilBertForSequenceClassification.from_pretrained(model_name).to(device)
sigmoid = nn.Sigmoid()

Сразу же добавлю функцию классификации

In [16]:
def classify(inp, as_proba=True):
    inputs = clf_tokenizer(inp, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = clf(**inputs).logits
    if as_proba:
        logits = sigmoid(logits)
    return logits[:, 1].cpu().numpy()

multiclassify = np.vectorize(classify)

На тему того, как можно генерироать более положительные отзывы, я не придумал ничего лучше, чем модифицировать промпт. 

1. У меня была мысль с более хитрым бим сёрчем - можно было бы оставлять такие опции, которые не только более вероятны, но и имеют положительную тональность. Это можно было бы сделать даже при помощи той же гпт2, если взять другую архитектуру, но пришлось бы переделывать функцию генерации, это во-первых не очень очевидно, во-вторых будет не совсем эффективно вроде как, потокенная генерация дело долгое. В общем, показалось, что это просто слишком сложно для самого начала задания, ответ наверное должен быть проще. Если бы я мог решить эту проблему на этапе бим сёрча, всё задание ниже не имело бы смысла

2. Была мысль с тупой генерацией как можно большего числа примеров, затем осталось бы выделить из них положительные и радоваться жизни, но это не непосредственно модификация модели

3. Файн-тюн это тоже опция, конечно, но это фактически попытка решить нашу задачу, тоже не то

4. Модифицировать промпт это тоже не совсем понятная вещь, потому что прописать можно много всего разного. Я остановился на 'Good Review', он вполне себе даёт результаты, как будет видно ниже

In [28]:
seed_everything(69)

n_samples = 30
samples = np.empty((n_samples, 3), dtype=object)
for i in trange(n_samples):
    for j, guidance in enumerate([None, "negative", "positive"]):
        samples[i, j] = generate_from_input(naive_guidance=guidance, **generation_config)

scores = multiclassify(samples)
scores.mean(0)

array([0.53372973, 0.20303504, 0.68123496], dtype=float32)

Тут уже с вероятностями классов. Видно, что третий столбец, именно там были положительные промпты, оказывается в среднем более позитивным, значит с этим и будем работать

In [31]:
seed_everything(69)

n_samples = 10000
samples = np.empty(n_samples, dtype=object)
for i in trange(n_samples):
    samples[i] = generate_from_input(naive_guidance="positive", **generation_config)

  0%|          | 0/10000 [00:00<?, ?it/s]

Дальше нужно каждый из них оценить, благо это не очень долго. Средний реворд действительно больше 0, всё в целом нормально, разнообразие тоже посчитано внизу

In [17]:
reward = np.vectorize(lambda x: classify(x, False))(samples)
reward.mean(), reward.max()

(0.46090737, 2.9355285)

In [19]:
token_entropy(samples, clf_tokenizer)

Evaluating:   0%|          | 0/10000 [00:00<?, ?it/s]

4.73228185758214

Все отзывы и реворды я на всякий случай сохранил, так что можно не прогонять всё, что выше, а сразу переходить к следующему пункту

In [17]:
samples = np.load("samples.npy", allow_pickle=True)
reward = np.load("rewards.npy")

Я возьму за хорошие пары все те, у которых реворд больше 2, кажется, что они достаточно положительные

In [18]:
winner = reward > 2
loser = (0 < reward) & (reward < 2)

winner_samples, loser_samples = samples[winner], samples[loser]
winner_reward, loser_reward = reward[winner], reward[loser]

Останется только собрать из них датасет и можно будет приступать к обучению

In [19]:
def generate_dataset(n_samples):
    
    dpo_dataset = defaultdict(list)

    winner_sample = np.random.choice(winner_samples, size=n_samples)
    loser_sample = np.random.choice(loser_samples, size=n_samples)
    prompts = ["Good Review:"]*n_samples

    dpo_dataset["prompt"] = prompts
    dpo_dataset["chosen"] = winner_sample  
    dpo_dataset["rejected"] = loser_sample

    dpo_dataset = Dataset.from_dict(dpo_dataset)
    dpo_dataset = dpo_dataset.map(batched=True)
    
    return dpo_dataset

In [20]:
# seed_everything(69)

dataset = generate_dataset(int(1e3))
dataset = dataset.train_test_split(test_size=0.2)
dataset.save_to_disk("dataset.hf")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

Всю логику обучения я оформил в отдельном классе, потому что вызывать её придётся очень часто, можно с этим ознакомиться там. Вкратце: загружаем датасет, копируем модельки, запускаем `DPOTrainer` с нужными параметрами, обучаем, сохраняем метрики

#### 1.2 Сравнение лоссов

In [7]:
from pipeline import Pipeline

pipe = Pipeline()
pipe.train(loss_type="hinge")
pipe.evaluate("hinge", n_samples=100)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
1,No log,1.000000,0.000000,0.000000,0.000000,0.000000,-121.575462,-72.020729,-42.489925,-37.603886
2,No log,1.000000,0.000000,0.000000,0.000000,0.000000,-158.835388,-71.110535,-43.824852,-36.914539
3,No log,1.000000,0.000000,0.000000,0.000000,0.000000,-165.572632,-72.145340,-44.334946,-37.273350


Generating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

3.783983918485955

In [5]:
# hinge

for _ in range(3):
    print(pipe.generate_from_input("This movie"))

This movie is a lot of fun to watch. The acting is good, the story is well written, and the characters are likable. I think this is one of the best movies I've seen in a long time.
This movie is one of the best movies I've seen in a long time. The acting is great, the story is well written, and the characters are likable. I highly recommend this movie to anyone who is looking for a good movie.
This movie is a great example of how to make a good horror movie. The acting is great, the story is well written, and the characters are likable. I highly recommend this movie to anyone who is looking for a horror film.


In [8]:
from pipeline import Pipeline

pipe = Pipeline()
pipe.train(loss_type="sigmoid")
pipe.evaluate("sigmoid", n_samples=100)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
1,No log,0.693147,0.000000,0.000000,0.000000,0.000000,-121.575447,-72.020729,-42.489937,-37.603867
2,No log,0.693147,0.000000,0.000000,0.000000,0.000000,-158.835342,-71.110527,-43.824833,-36.914505
3,No log,0.693147,0.000000,0.000000,0.000000,0.000000,-165.572571,-72.145332,-44.334923,-37.273323


Generating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

3.783983918485955

In [4]:
# sigmoid

for _ in range(3):
    print(pipe.generate_from_input("This movie"))

This movie great this movie. It is great. The acting is good, the story is well told, and the special effects are very good. This is one of the best movies I have seen in a long time.
This movie great this movie. It is great. The acting is good, the story is well told, and the characters are well developed. This is one of the best movies I have seen in a long time.
This movie great this movie. It is great. The acting is good, the story is well told, and the special effects are very good. This is one of the best movies I have seen in a long time.


In [9]:
import pandas as pd
import numpy as np

all_dfs = ["sigmoid", "hinge"]
summary = pd.concat([pd.read_csv(f"{df}.csv") for df in all_dfs])
summary = summary.groupby("experiment").mean("reward").sort_values("reward")
summary.loc["best"] = [2.935, 4.732]
summary.T

experiment,hinge,sigmoid,best
reward,2.802011,2.834603,2.935
diversity,3.741267,3.731363,4.732


Как можно заметить, между лоссами нет принципиальной разницы, как обычно её нет при обучении свм и логрега. В общем и целом можно заметить, что средний реворд действительно вырос и очень даже неплохо, но при этом сильно пострадало разнообразие текстов. Можно заметить, что они все почти одинаковые, это может быть не очень хорошо, но быть может и обучение было проведено тоже не идеально

### Часть 2

Здесь нужно внимательно посмотреть в статью и увидеть там 2 вещи

1. Лосс в обобщённом виде, где $f'$ это производная произвольной дивергенции, в `DPOTrainer` это обратная КЛ-дивергенция, её мы и будем менять

$$
\mathcal{L}(\boldsymbol{\theta}, \mathcal{D})=\mathbb{E}_{\left(x, y_w, y_l\right) \sim \mathcal{D}}\left[-\log \sigma\left(\beta f^{\prime}\left(\frac{\pi_{\boldsymbol{\theta}}\left(y_w | x\right)}{\pi_{\mathrm{ref}}\left(y_w | x\right)}\right)-\beta f^{\prime}\left(\frac{\pi_{\boldsymbol{\theta}}\left(y_l | x\right)}{\pi_{\mathrm{ref}}\left(y_l  | x\right)}\right)\right)\right] .
$$

2. Это табличка, где все производные уже любезно подсчитаны за нас, дальше остаётся только это дело закодить и вставить в класс

$$
\begin{array}{lllc}
\hline f \text {-divergence } & \boldsymbol{f}(\boldsymbol{u}) & \boldsymbol{f}^{\prime}(\boldsymbol{u}) & 0 \notin \text { Domain of } \boldsymbol{f}^{\prime}(\boldsymbol{u}) \\
\hline \alpha \text {-divergence }(\alpha \in(0,1)) & \left(u^{1-\alpha}-(1-\alpha) u-\alpha\right) /(\alpha(\alpha-1)) & \left(1-u^{-\alpha}\right) / \alpha & \checkmark \\
\text { Reverse KL }(\alpha=0) & u \log u & \log u+1 & \checkmark \\
\text { Forward KL }(\alpha=1) & -\log u & -1 / u & \checkmark \\
\text { JS-divergence } & u \log u-(u+1) \log ((u+1) / 2) & \log (2 u /(1+u)) & \checkmark \\
\hline \text { Total Variation } & \frac{1}{2}|u-1| & u>1 ? \frac{1}{2}:-\frac{1}{2} & \mathbf{x} \\
\text { Chi-squared } & (u-1)^2 & 2(u-1) & \mathbf{X} \\
\hline
\end{array}
$$

Посколько для обучения любой сети нам нужен только её лосс, нужно это же место и найти. Он считается в методе `dpo_loss`, ниже конкретные места, которые надо поменять:

```python
pi_logratios = policy_chosen_logps - policy_rejected_logps
if reference_free:
    ref_logratios = 0
else:
    ref_logratios = reference_chosen_logps - reference_rejected_logps
```

на

```python
pi_ratios = self.divergence(policy_chosen_logps, policy_rejected_logps)
ref_ratios = self.divergence(reference_chosen_logps, reference_rejected_logps)
```

В классе уже имплементирован случай, когда считается RKL-div, там уже удобно сделано, ведь там есть логарифм, остаётся их только повычитать. В случае же обычной дивергенции нужно иметь дело с отношением и экспонентой, это взрывает градиенты. Я обошёл это как мог, но взрыв всё равно есть, поэтому я принудительно их клипаю и делаю апкаст . Как выглядят функции можно посмотреть в `utils.py`, то, как я это встроил - в `pipeline.py`

Проверим, что у нас получается примерно то, что надо. Имеем в виду, что при экстремальных $\alpha$ альфа-дивергенция должна быть похожа на КЛ

In [1]:
from utils import (alpha_divergence, KL_divergence, RKL_divergence, JS_divergence)
import torch

a = torch.tensor([2, 3, 4, 5])
b = torch.tensor([1, 2, 3, 10])

print(RKL_divergence(a, b))
print(alpha_divergence(a, b, alpha=0.001))
print(alpha_divergence(a, b, alpha=0.5))
print(alpha_divergence(a, b, alpha=1-1e-10))
print(KL_divergence(a, b))
print(JS_divergence(a, b))

tensor([ 1,  1,  1, -5])
tensor([ 0.9995,  0.9995,  0.9995, -5.0125], dtype=torch.float64)
tensor([  0.7869,   0.7869,   0.7869, -22.3650], dtype=torch.float64)
tensor([   0.6321,    0.6321,    0.6321, -147.4132], dtype=torch.float64)
tensor([   0.6321,    0.6321,    0.6321, -147.4132], dtype=torch.float64)
tensor([ 0.3799,  0.3799,  0.3799, -4.3136], dtype=torch.float64)


Ну вроде всё примерно так, а значит, можно запускать цикл и идти пить чай

In [2]:
import numpy as np
from functools import partial

divs = [
    RKL_divergence, JS_divergence, KL_divergence
]
divs += [
    partial(lambda x, y, alpha: alpha_divergence(x, y, alpha=alpha), alpha=alpha)
    for alpha in [0.1, 0.3, 0.5, 0.7, 0.9]
]
names = ["rkl", "js", "kl", "a_1", "a_3", "a_5", "a_7", "a_9"]

In [ ]:
from pipeline import Pipeline
from tqdm.auto import tqdm
from IPython.display import clear_output
import gc

for div, name in zip(divs, names):

    try:
        pipe = Pipeline()
        pipe.train(loss_type="hinge", divergence=div)
        pipe.evaluate(name, n_samples=100)
        torch.cuda.empty_cache()
        gc.collect()
        clear_output(True)
    except:
        pass

In [ ]:
!tar chvfz notebook.tar.gz *

Теперь соберём всё воедино и построим график

In [30]:
import pandas as pd
import numpy as np

all_dfs = ["kl", "rkl", "js", "a_3", "a_5", "a_7", "a_9"]
summary = pd.concat([pd.read_csv(f"{df}.csv") for df in all_dfs])
summary = summary.groupby("experiment").mean("reward").sort_values("reward")

In [ ]:
!tar 

In [31]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

summary["sigma_reward"] = sigmoid(summary.reward)

In [32]:
summary.T

experiment,a_9,a_7,a_5,kl,a_3,js,rkl
reward,2.385917,2.454308,2.515007,2.518397,2.650520,2.658916,2.710398
diversity,4.300991,4.476865,4.250743,4.334102,4.227138,4.264548,3.952406
sigma_reward,0.915747,0.920876,0.925187,0.925421,0.934043,0.934558,0.937637


In [33]:
summary = summary.reset_index()

In [36]:
import plotly.graph_objects as go

z = np.polyfit(summary["sigma_reward"], summary["diversity"], 2)
f = np.poly1d(z)

x_new = np.linspace(0.915, 0.94, 50)
y_new = f(x_new)

trace1 = go.Scatter(
    x=summary["sigma_reward"],
    y=summary["diversity"],
    text=summary["experiment"],
    mode='markers+text',
    marker=dict(size=8),
    marker_symbol="x",
    marker_color="black",
    showlegend=False,
    name="Experiments"
)
trace2 = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    showlegend=False,
    line = dict(dash='dash')
)

data = [trace1, trace2]
fig = go.Figure(data=data)
fig.update_traces(textposition="bottom right")
fig.update_layout(
    height=500, width=500,
    title="Reward-Entropy Tradeoff",
    xaxis_title="Reward", yaxis_title="Entropy"
)
fig.write_html("diversities.html")
#fig.show() # плотли иногда не рисуется

<img src="https://media.discordapp.net/attachments/674191702906503199/1183868305069588662/image.png?ex=6589e647&is=65777147&hm=ba67361e019a063a44c9e9416a1accbaceacc33605a0690e158c200e39e44d9b&=&format=webp&quality=lossless&width=631&height=654" style="height: 400px">

График получился не такой красивый, как в статье, и оправданий может быть масса. 

Во-первых, параметры, вполне возможно, что у меня набор не совсем оптимальный. Я старался скопировать всё так, как там написано, но всё равно разница есть. Как минимум у них gpt4, это другое, как говорится.

Во-вторых, у меня всего лишь один запуск, я не успел больше, у меня на это 4 часа после работы (. Если делать нормальный честный эксперимент, то нужно делать несколько запусков и брать среднее, в идеале рисовать errorbar. Я думаю, что в статье так и сделали, потому что больно ровная у них картинка, но может быть и я где-то накосячил в функциях. Хотя простенький тест сверху показал, что должно быть ок

Полифит показывает примерно то же самое, но тоже оказался чуть смещён. У меня реворды и дайвёрсити в целом чуть больше, ну опять же у меня чуть-чуть другие условия. В целом видим то же самое - с ростом альфы реворд падает, но тексты более разные. В целом какую дивергенцию ни возьми всё получается, более-менее

### Часть 3

Вот тут мне уже не хватает математической подготовки. Ну и не совсем понятно, что считать хорошим результатом. Я бы сказал, что здесь нужно максимизировать и разнообразие, и реворды одновременно, но как именно это можно было бы сделать, я не представляю. Разве что попробовать покрутить гиперпараметры, попридумывать более разнообразных промптов и так далее. Может быть есть какие-то спосообы обогатить модель после её обучения - накидать каких-нибудь аугментаций к новым генерациям модели и попробовать сделать её разнообразнее. Как минимум можно разбавить синонимами, это немного, но хоть что-то. Ещё я читал про другие методы, типа PPO, но они тоже уже придуманы. Может быть есть какие-то другие виды дивергенций, но я кроме КЛ ни одной не знаю